In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Creación de datos clínicos simulados

In [ ]:
np.random.seed(42)
n_registros = 500

# Generar datos para múltiples pacientes
pacientes = ['P001', 'P002', 'P003', 'P004', 'P005']
estados = ['Normal', 'Alerta', 'Crítico']

df = pd.DataFrame({
    'paciente_id': np.random.choice(pacientes, n_registros),
    'timestamp': pd.date_range(start='2025-01-01', periods=n_registros, freq='5min'),
    'HR': np.random.randint(50, 140, n_registros),
    'SBP': np.random.randint(90, 180, n_registros),
    'DBP': np.random.randint(60, 110, n_registros),
    'SpO2': np.random.randint(88, 100, n_registros),
    'temperatura': np.random.uniform(36.0, 39.5, n_registros),
    'estado_clinico': np.random.choice(estados, n_registros, p=[0.6, 0.3, 0.1])
})

print("=== Dataset clínico ===")
print(df.head(10))
print(f"\nTotal de registros: {len(df)}")

## 1. describe() - Resumen estadístico completo

In [ ]:
# Resumen estadístico básico
print("=== Resumen estadístico de signos vitales ===")
resumen = df[['HR', 'SBP', 'DBP', 'SpO2', 'temperatura']].describe()
print(resumen)

# Incluir percentiles adicionales
print("\n=== Con percentiles adicionales (5%, 95%, 99%) ===")
resumen_detallado = df[['HR', 'SBP', 'DBP', 'SpO2']].describe(
    percentiles=[0.05, 0.25, 0.5, 0.75, 0.95, 0.99]
)
print(resumen_detallado)

## 2. value_counts() - Conteo de categorías

In [ ]:
# Conteo de estados clínicos
print("=== Distribución de estados clínicos ===")
conteo_estados = df['estado_clinico'].value_counts()
print(conteo_estados)

print("\n=== En porcentajes ===")
conteo_porcentaje = df['estado_clinico'].value_counts(normalize=True) * 100
print(conteo_porcentaje.round(2))

# Conteo por paciente
print("\n=== Registros por paciente ===")
registros_paciente = df['paciente_id'].value_counts()
print(registros_paciente)

## 3. nunique() - Valores únicos

In [ ]:
# Contar valores únicos en cada columna
print("=== Número de valores únicos por columna ===")
valores_unicos = df.nunique()
print(valores_unicos)

print("\n=== Análisis de variabilidad ===")
print(f"Pacientes únicos: {df['paciente_id'].nunique()}")
print(f"Estados clínicos únicos: {df['estado_clinico'].nunique()}")
print(f"Valores únicos de HR: {df['HR'].nunique()}")
print(f"Valores únicos de SBP: {df['SBP'].nunique()}")

## 4. corr() - Matriz de correlación

In [ ]:
# Calcular correlaciones entre signos vitales
print("=== Matriz de correlación ===")
correlaciones = df[['HR', 'SBP', 'DBP', 'SpO2', 'temperatura']].corr()
print(correlaciones.round(3))

# Correlaciones específicas
print("\n=== Correlaciones específicas ===")
print(f"HR vs SBP: {df['HR'].corr(df['SBP']):.3f}")
print(f"HR vs SpO2: {df['HR'].corr(df['SpO2']):.3f}")
print(f"SBP vs SpO2: {df['SBP'].corr(df['SpO2']):.3f}")
print(f"SBP vs DBP: {df['SBP'].corr(df['DBP']):.3f}")

## 5. Visualización de matriz de correlación

In [ ]:
# Heatmap de correlaciones
plt.figure(figsize=(10, 8))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, fmt='.3f')
plt.title('Matriz de Correlación - Signos Vitales', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Heatmap de correlaciones generado")

## 6. Identificación de patrones globales

In [ ]:
# Análisis de rangos y variaciones
print("=== Patrones globales ===")

for columna in ['HR', 'SBP', 'DBP', 'SpO2']:
    media = df[columna].mean()
    mediana = df[columna].median()
    std = df[columna].std()
    rango = df[columna].max() - df[columna].min()
    cv = (std / media) * 100  # Coeficiente de variación
    
    print(f"\n{columna}:")
    print(f"  Media: {media:.2f}")
    print(f"  Mediana: {mediana:.2f}")
    print(f"  Desviación estándar: {std:.2f}")
    print(f"  Rango: {rango:.2f} (min: {df[columna].min():.2f}, max: {df[columna].max():.2f})")
    print(f"  Coeficiente de variación: {cv:.2f}%")

## 7. Tablas resumen por paciente

In [ ]:
# Resumen estadístico por paciente
print("=== Resumen por paciente ===")

resumen_pacientes = df.groupby('paciente_id').agg({
    'HR': ['mean', 'min', 'max', 'std'],
    'SBP': ['mean', 'min', 'max', 'std'],
    'SpO2': ['mean', 'min', 'max', 'std']
}).round(2)

print(resumen_pacientes)

In [ ]:
# Tabla resumen más compacta
print("\n=== Tabla resumen compacta por paciente ===")

resumen_compacto = df.groupby('paciente_id').agg({
    'HR': 'mean',
    'SBP': 'mean',
    'DBP': 'mean',
    'SpO2': 'mean',
    'temperatura': 'mean',
    'estado_clinico': lambda x: x.value_counts().index[0]  # Estado más frecuente
}).round(2)

resumen_compacto.columns = ['HR_promedio', 'SBP_promedio', 'DBP_promedio', 
                              'SpO2_promedio', 'Temp_promedio', 'Estado_frecuente']
print(resumen_compacto)

## 8. Análisis por estado clínico

In [ ]:
# Comparar signos vitales según estado clínico
print("=== Signos vitales por estado clínico ===")

resumen_estado = df.groupby('estado_clinico').agg({
    'HR': ['mean', 'std'],
    'SBP': ['mean', 'std'],
    'SpO2': ['mean', 'std']
}).round(2)

print(resumen_estado)

## 9. Tabla dinámica (pivot table)

In [ ]:
# Crear tabla dinámica: paciente vs estado clínico
print("=== Tabla dinámica: Conteo de estados por paciente ===")

tabla_dinamica = pd.pivot_table(
    df,
    values='HR',
    index='paciente_id',
    columns='estado_clinico',
    aggfunc='count',
    fill_value=0
)

print(tabla_dinamica)

# Tabla dinámica con promedios
print("\n=== Tabla dinámica: HR promedio por paciente y estado ===")

tabla_hr_promedio = pd.pivot_table(
    df,
    values='HR',
    index='paciente_id',
    columns='estado_clinico',
    aggfunc='mean'
).round(2)

print(tabla_hr_promedio)

## 10. Resumen estadístico avanzado personalizado

In [ ]:
# Crear función personalizada para resumen
def resumen_clinico(serie):
    """Genera resumen clínico personalizado"""
    return pd.Series({
        'count': len(serie),
        'mean': serie.mean(),
        'median': serie.median(),
        'std': serie.std(),
        'min': serie.min(),
        'max': serie.max(),
        'range': serie.max() - serie.min(),
        'q25': serie.quantile(0.25),
        'q75': serie.quantile(0.75),
        'iqr': serie.quantile(0.75) - serie.quantile(0.25)
    })

print("=== Resumen clínico personalizado ===")
resumen_hr = resumen_clinico(df['HR'])
print("\nHeart Rate:")
print(resumen_hr.round(2))

print("\nSystolic Blood Pressure:")
resumen_sbp = resumen_clinico(df['SBP'])
print(resumen_sbp.round(2))

## 11. Reporte completo por paciente

In [ ]:
# Generar reporte completo para cada paciente
print("=== REPORTE COMPLETO POR PACIENTE ===")

for paciente in df['paciente_id'].unique():
    datos_paciente = df[df['paciente_id'] == paciente]
    
    print(f"\n{'='*60}")
    print(f"PACIENTE: {paciente}")
    print(f"{'='*60}")
    print(f"Total de registros: {len(datos_paciente)}")
    
    print("\nSignos vitales promedio:")
    print(f"  HR: {datos_paciente['HR'].mean():.1f} bpm")
    print(f"  SBP: {datos_paciente['SBP'].mean():.1f} mmHg")
    print(f"  DBP: {datos_paciente['DBP'].mean():.1f} mmHg")
    print(f"  SpO2: {datos_paciente['SpO2'].mean():.1f}%")
    print(f"  Temperatura: {datos_paciente['temperatura'].mean():.1f}°C")
    
    print("\nDistribución de estados:")
    estados_paciente = datos_paciente['estado_clinico'].value_counts()
    for estado, conteo in estados_paciente.items():
        porcentaje = (conteo / len(datos_paciente)) * 100
        print(f"  {estado}: {conteo} ({porcentaje:.1f}%)")

## Resumen

### Funciones estadísticas principales:

1. **describe()**: Resumen estadístico completo (count, mean, std, min, quartiles, max)
2. **value_counts()**: Conteo de frecuencias para variables categóricas
3. **nunique()**: Número de valores únicos en cada columna
4. **corr()**: Matriz de correlación entre variables numéricas

### Aplicaciones clínicas:
- Identificar rangos normales y anormales de signos vitales
- Detectar correlaciones entre parámetros fisiológicos
- Comparar patrones entre pacientes
- Generar reportes resumen automatizados

### Interpretación de correlaciones:
- **0.7 - 1.0**: Correlación fuerte
- **0.3 - 0.7**: Correlación moderada
- **0.0 - 0.3**: Correlación débil
- **Negativa**: Relación inversa entre variables